<a href="https://colab.research.google.com/github/Annapoornaaradhya/Ask-IPC/blob/main/Chatbot_on_Indian_Penal_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

from langchain import PromptTemplate, LLMChain
from langchain.llms import Cohere
from pydantic import BaseModel
from typing import Optional
import wikipediaapi

import getpass
import wikipediaapi
from pydantic import BaseModel
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from IPython.display import display
import ipywidgets as widgets

# 1. Setup Cohere LLM with API Key
api_key = getpass.getpass('Enter Cohere API Key: ')
llm = Cohere(cohere_api_key=api_key, model="command")

# 2. Fetch IPC summary from Wikipedia
def get_ipc_content():
    wiki = wikipediaapi.Wikipedia(user_agent="IPCChatbot/1.0", language='en')
    page = wiki.page("Indian_Penal_Code")
    if not page.exists():
        raise Exception("IPC page not found!")
    return page.text[:5000]

ipc_content = get_ipc_content()

# 3. Define structured response model
class IPCResponse(BaseModel):
    section: str = None
    explanation: str

# 4. Prompt template
prompt = PromptTemplate(
    input_variables=["ipc_content", "question"],
    template="""You are a legal assistant. Refer to:
{ipc_content}
Question: {question}
Give a detailed answer, mentioning section if applicable."""
)

# 5. Function to get IPC response
def get_response(question):
    formatted = prompt.format(ipc_content=ipc_content, question=question)
    reply = llm.predict(formatted)

    if "Section" in reply:
        section = reply.split('Section')[1].split(':')[0].strip()
        explanation = reply.split(':', 1)[-1].strip()
    else:
        section, explanation = None, reply.strip()

    return IPCResponse(section=section, explanation=explanation)

# 6. Display chatbot response
def show_response(resp):
    print(f"Section: {resp.section or 'N/A'}")
    print(f"Explanation: {resp.explanation}")

# 7. Widgets for user interaction
text_box = widgets.Text(placeholder='Ask about IPC', description='You:')
button = widgets.Button(description='Ask', icon='legal')

def on_click(b):
    try:
        resp = get_response(text_box.value)
        show_response(resp)
    except Exception as e:
        print(f"Error: {e}")

button.on_click(on_click)
display(text_box, button)


Enter Cohere API Key: ··········


Text(value='', description='You:', placeholder='Ask about IPC')

Button(description='Ask', icon='legal', style=ButtonStyle())

Section: 378 of the IPC. It defines theft as follows
Explanation: Whoever steals, or commits theft, shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both.

For the purposes of this section, "theft" means:

(a) the dishonest appropriation of property owned by another person, with the intention of permanently depriving the true owner of it; or
(b) the dishonest sale, disposal, transfer, or conversion of property owned by another person, or of his entitlement to it, knowing that the property would disappear, or the entitlement would cease to exist, into the possession of one person, and disposed of or transferred to another person,

Section 411 outlines the definition of "dishonest" in the context of theft as "dealing with another's property in a way that amounts to a wrongful loss to him."

Theft can be prosecuted in cases where the accused intentionally steals, or commits theft of, anything capable of being stole